In [65]:
import os
import re
import string
import json
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW , BertForSequenceClassification ,RobertaTokenizer, RobertaForSequenceClassification
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
train_data = pd.read_csv('TRAIN_DATA.csv')
test_data = pd.read_csv('TEST_DATA.csv')
val_data = pd.read_csv('VAL_DATA.csv')

In [3]:
train_data.head()

,Sentence,Emotion,Length
0,My favourite food is anything I didn't have to...,6,59
1,"Now if he does off himself, everyone will thin...",6,112
2,WHY THE FUCK IS BAYLESS ISOING,0,30
3,To make her feel threatened,2,27
4,Dirty Southern Wankers,0,22


In [17]:
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments

# Load tokenizer and model
num_labels = 7  # Update this with the actual number of classes
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)

# Define the tokenization function
def tokenize_function(sentences):
    return tokenizer(sentences, padding='max_length', truncation=True, max_length=128)

# Example train_data DataFrame
import pandas as pd

# Create your DataFrame (replace this with your actual data)
train_data = pd.DataFrame({
    'Sentence': [
        "My favourite food is anything I didn't have to cook.",
        "Now if he does off himself, everyone will think he was just acting out.",
        "WHY THE FUCK IS BAYLESS ISOING",
        "To make her feel threatened",
        "Dirty Southern Wankers"
    ],
    'Emotion': [6, 6, 0, 2, 0]
})

# Tokenize the sentences
train_encodings = tokenize_function(train_data['Sentence'].tolist())

# Create a custom dataset class
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Prepare the dataset
train_dataset = EmotionDataset(train_encodings, train_data['Emotion'].tolist())
test_dataset = EmotionDataset(train_encodings, test_data['Emotion'].tolist())

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.0590621630350747, metrics={'train_runtime': 2.9799, 'train_samples_per_second': 5.034, 'train_steps_per_second': 1.007, 'total_flos': 986710752000.0, 'train_loss': 2.0590621630350747, 'epoch': 3.0})

In [21]:
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Ensure idx is within range
        if idx >= len(self.labels):
            raise IndexError("Index out of range for labels.")
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# Check lengths before creating the datasets
train_encodings = tokenize_function(train_data['Sentence'].tolist())
train_labels = train_data['Emotion'].tolist()
test_encodings = tokenize_function(test_data['Sentence'].tolist())
test_labels = test_data['Emotion'].tolist()

# Ensure lengths match
assert len(train_encodings['input_ids']) == len(train_labels), "Training encodings and labels must have the same length."
assert len(test_encodings['input_ids']) == len(test_labels), "Testing encodings and labels must have the same length."

# Create datasets
train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)

# Now create Trainer with eval_dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Your training dataset
    eval_dataset=test_dataset      # Your evaluation dataset
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

# Save the trained model
model.save_pretrained('./saved_roberta_model')
tokenizer.save_pretrained('./saved_roberta_model')


Step,Training Loss


('./saved_roberta_model\\tokenizer_config.json',
 './saved_roberta_model\\special_tokens_map.json',
 './saved_roberta_model\\vocab.json',
 './saved_roberta_model\\merges.txt',
 './saved_roberta_model\\added_tokens.json')

In [27]:
import torch

# After training
trainer.evaluate()

# Check model predictions (optional)
predictions = trainer.predict(test_dataset)

# Convert NumPy array to PyTorch tensor
predictions_tensor = torch.tensor(predictions.predictions)

# Get predicted labels
predicted_labels = torch.argmax(predictions_tensor, dim=1)

# If you want to convert the predicted labels to a list for further processing
predicted_labels_list = predicted_labels.tolist()

# Optional: Display the predicted labels
print(predicted_labels_list)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [33]:
# Assuming train_data is your training dataset
from transformers import Trainer, TrainingArguments


val_dataset = EmotionDataset(train_encodings, val_data['Emotion'].tolist())

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                        # the instantiated 🤗 Transformers model to be trained
    args=training_args,                 # training arguments, defined above
    train_dataset=train_dataset,        # training dataset
    eval_dataset=val_dataset            # evaluation dataset
)

# Start training
trainer.train()


Step,Training Loss


TrainOutput(global_step=3, training_loss=1.8420368830362956, metrics={'train_runtime': 2.2205, 'train_samples_per_second': 6.755, 'train_steps_per_second': 1.351, 'total_flos': 986710752000.0, 'train_loss': 1.8420368830362956, 'epoch': 3.0})

In [39]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 3
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [47]:
target_cols = [col for col in train_data.columns if col not in ['Sentence']]
target_cols

['Emotion']

In [53]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df['Sentence']  # Access the 'Sentence' column for text
        self.tokenizer = tokenizer
        self.targets = df['Emotion'].values  # Access the 'Emotion' column for targets
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'  # Return PyTorch tensors directly
        )
        
        # Extract input ids and attention mask from the inputs
        ids = inputs['input_ids'].squeeze(0)  # Remove the batch dimension
        mask = inputs['attention_mask'].squeeze(0)  # Remove the batch dimension
        
        return {
            'ids': ids,
            'mask': mask,
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [55]:
train_dataset = BERTDataset(train_data, tokenizer, MAX_LEN)
valid_dataset = BERTDataset(val_data, tokenizer, MAX_LEN)

In [57]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)

In [73]:
# Assuming you have a DataFrame 'train_df' with your training data
max_len = 128  # or any length you want
batch_size = 16  # adjust based on your memory capacity

# Create the dataset
train_dataset = BERTDataset(test_data, tokenizer, max_len)

# Create the DataLoader
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [83]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AdamW

# Define the BERTDataset class
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df['Sentence']  # Adjust according to your DataFrame's column name
        self.tokenizer = tokenizer
        self.targets = df['Emotion'].values  # Adjust to match the correct target column name
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)  # Use long for targets
        }

# Define the model class
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
        self.fc = torch.nn.Linear(768, 7)  # Adjust output size based on your classes (0-6 for 7 classes)
    
    def forward(self, ids, mask, token_type_ids):
        outputs = self.roberta(ids, attention_mask=mask, token_type_ids=token_type_ids)
        features = outputs[1]  # Get the pooled output
        output = self.fc(features)
        return output

# Prepare your DataFrame (assuming 'train_df' is your DataFrame with 'Sentence' and 'Emotion')
data = {
    'Sentence': [
        "My favourite food is anything I didn't have to cook.",
        "Now if he does off himself, everyone will think he is a coward.",
        "WHY THE FUCK IS BAYLESS ISOING",
        "To make her feel threatened",
        "Dirty Southern Wankers"
    ],
    'Emotion': [0, 1, 2, 3, 4]  # Adjust according to your emotion classes, ensuring they match the number of classes
}
train_df = pd.DataFrame(data)

# Define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Hyperparameters
max_len = 128
batch_size = 16
learning_rate = 2e-5
num_epochs = 3

# Create the dataset and DataLoader
train_dataset = BERTDataset(train_df, tokenizer, max_len)
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model and move it to the appropriate device
model = BERTClass()
model.to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch in dataloader:
        ids = batch['ids'].to(device)
        mask = batch['mask'].to(device)
        targets = batch['targets'].to(device)

        # Forward pass
        outputs = model(ids, mask)

        # Compute the loss
        loss = torch.nn.functional.cross_entropy(outputs, targets)  # Ensure targets are long

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [69]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
        self.dropout = torch.nn.Dropout(0.3)  # Dropout layer for regularization
        self.fc = torch.nn.Linear(self.roberta.config.hidden_size, 5)  # Adjust output layer to match number of classes
    
    def forward(self, ids, mask):
        # Get the last hidden state and pooled output from RoBERTa
        outputs = self.roberta(ids, attention_mask=mask)
        features = outputs.last_hidden_state[:, 0, :]  # Take the first token (CLS token)
        
        # Apply dropout and then the fully connected layer
        features = self.dropout(features)
        output = self.fc(features)
        return output

# Instantiate the model and move it to the appropriate device
model = BERTClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTClass(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer